# Extracción de datos Raw

In [ ]:
# Extracción de datos Raw con filtros en la API
import os
import time
import logging
import requests
import pandas as pd
from urllib.parse import urlencode

# 📂 Rutas base
BASE_PATH = r"C:\Users\POTENCIA\OneDrive - POTENCIA\Documents\TAREA_ENTIDADES"
RAW_PATH = os.path.join(BASE_PATH, "data", "00_raw", "secop")
LOG_PATH = os.path.join(BASE_PATH, "data", "logs")

os.makedirs(RAW_PATH, exist_ok=True)
os.makedirs(LOG_PATH, exist_ok=True)

# 📝 Configuración de logging
logging.basicConfig(
    filename=os.path.join(LOG_PATH, "extraccion_secop.log"),
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

# 🌐 API Socrata
BASE_URL = "https://www.datos.gov.co/resource/jbjy-vk9h.json"

# ⚙️ Parámetros
CHUNK_SIZE = 5000
MAX_RETRIES = 3
SLEEP_TIME = 1

# ✅ Columnas que quieres conservar
COLUMNS = [
    "nombre_entidad", "nit_entidad", "codigo_entidad", "orden", "sector",
    "id_contrato", "estado_contrato", "tipo_de_contrato",
    "modalidad_de_contratacion", "codigo_de_categoria_principal",
    "descripcion_del_proceso", "fecha_de_inicio_del_contrato",
    "fecha_de_fin_del_contrato", "tipodocproveedor", "documento_proveedor",
    "codigo_proveedor", "proveedor_adjudicado", "origen_de_los_recursos",
    "valor_del_contrato", "c_digo_bpin", "urlproceso",
    "presupuesto_general_de_la_nacion_pgn", "sistema_general_de_participaciones",
    "sistema_general_de_regal_as",
    "recursos_propios_alcald_as_gobernaciones_y_resguardos_ind_genas_",
    "recursos_de_credito", "recursos_propios", "objeto_del_contrato"
]

def download_year(year):
    logging.info(f"🔎 Iniciando descarga para el año {year}")
    offset = 0
    all_chunks = []

    while True:
        params = {
            "$limit": CHUNK_SIZE,
            "$offset": offset,
            "$select": ", ".join(COLUMNS),
            "$where": f"orden='Nacional' AND tipo_de_contrato='Obra' "
                      f"AND codigo_de_categoria_principal like 'V1.72%' "
                      f"AND date_extract_y(fecha_de_inicio_del_contrato)={year}"
        }

        try:
            r = requests.get(BASE_URL, params=params)
            data = r.json()

            if not isinstance(data, list) or len(data) == 0:
                logging.info(f"Año {year} - fin de registros")
                break

            df = pd.DataFrame(data)

            # Conversión de columnas
            df['fecha_de_inicio_del_contrato'] = pd.to_datetime(df['fecha_de_inicio_del_contrato'], errors='coerce')
            df['valor_del_contrato'] = pd.to_numeric(df['valor_del_contrato'], errors='coerce')

            all_chunks.append(df)
            offset += CHUNK_SIZE

            logging.info(f"Año {year} - descargadas {offset} filas")
            time.sleep(SLEEP_TIME)

        except Exception as e:
            logging.error(f"Error año {year}: {e}")
            break

    return pd.concat(all_chunks, ignore_index=True) if all_chunks else pd.DataFrame()

# 🚀 Descargar todos los años y consolidar
all_years = []
for year in range(2019, 2026):
    df_year = download_year(year)
    if not df_year.empty:
        all_years.append(df_year)

df_final = pd.concat(all_years, ignore_index=True)

# 📊 Guardar en un solo Excel
output_file = os.path.join(RAW_PATH, "secop_obras_2019_2025.xlsx")
df_final.to_excel(output_file, index=False)

logging.info("Archivo consolidado guardado correctamente")